In [137]:
import requests
import pandas as pd
import time
import config
import pyarrow as pa
import numpy as np

In [89]:
paramaters = {'min_rank':'grand-champion','max_rank':'grand-champion','playlist':'ranked-doubles'}

head = {'Authorization': 'n0emNjRIkzyy4Z9Ii7HYYvCtani5AWT4T7HOcdrm'}

metadata = ['id', 'link', 'rocket_league_id', 'replay_title', 'map_code',
            'map_name', 'playlist_id', 'playlist_name', 'duration', 'overtime',
            'overtime_seconds', 'season', 'season_type', 'date', 'date_has_tz',
            'visibility', 'created', 'min_rank.tier', 'min_rank.division',
            'min_rank.name', 'min_rank.id', 'max_rank.tier', 'max_rank.division',
            'max_rank.name', 'max_rank.id', 'uploader.steam_id', 'uploader.name',
            'uploader.profile_url', 'uploader.avatar']

In [44]:
def get_players_csv(id: str) -> pd.DataFrame:

    try:
        df = pd.read_csv(f'https://ballchasing.com/dl/stats/players/{id}/{id}-players.csv', sep=';')
    except:
        print('Time Out')
        time.sleep(5)
        df = pd.read_csv(f'https://ballchasing.com/dl/stats/players/{id}/{id}-players.csv', sep=';')

    return df.T

In [43]:
def get_teams_csv(id: str) -> pd.DataFrame:

    try:
        df = pd.read_csv(f'https://ballchasing.com/dl/stats/teams/{id}/{id}-team-stats.csv', sep=';') # need to build in error handlding to the function here
    except:
        print('Time Out')
        time.sleep(5)
        df = pd.read_csv(f'https://ballchasing.com/dl/stats/teams/{id}/{id}-team-stats.csv', sep=';')
        
    return df.set_index('color').T

In [42]:
def add_team_data(df: pd.DataFrame) -> pd.DataFrame:

    blue_dict = {}
    orange_dict = {}
    
    for index, row in df.iterrows():
        
        time.sleep(4)
        teams_data = get_teams_csv(row['id'])
        
        blue_dict[index] = teams_data['blue']
        orange_dict[index] = teams_data['orange']

    df['blue_team'] = pd.Series(blue_dict)
    df['orange_team'] = pd.Series(orange_dict)
    
    return df

In [57]:
def add_player_data(df: pd.DataFrame) -> pd.DataFrame:

    bluep1_dict = {}
    bluep2_dict = {}
    orange1_dict = {}
    orange2_dict = {}
    
    for index, row in df.iterrows():
        
        time.sleep(4)
        player_data = get_players_csv(row['id'])
        
        bluep1_dict[index] = player_data[0]
        bluep2_dict[index] = player_data[1]
        orange1_dict[index] = player_data[2]
        orange2_dict[index] = player_data[3]

    df['blue_player1'] = pd.Series(bluep1_dict)
    df['blue_player2'] = pd.Series(bluep2_dict)
    df['orange_player1'] = pd.Series(orange1_dict)
    df['orange_player2'] = pd.Series(orange2_dict)
    
    return df

In [91]:
data = pd.DataFrame()

def new_data(criteria, auth, stuff) -> pd.DataFrame:

    api_request = requests.get('https://ballchasing.com/api/replays', params=criteria, headers=auth)
    infojson = api_request.json()

    df = pd.json_normalize(infojson, record_path=['list'], sep='.')

    df = df.drop(['blue.players','orange.players'], axis=1)

    for column_name in ['blue_team','blue_player1','blue_player2','orange_team','orange_player1','orange_player2']:
        df.insert(df.columns.size, column_name, pd.Series(None, dtype='object'))

    # add_team_data(df)
    # add_player_data(df)
    
    return df

**Data Cleaning** *OUTDATED*

* Data is received in a (trash) json format

* Everything is flattened at first except for the color.players column which still contains raw json text

* Each color of json text is flattened and then concatenated back into the main DataFrame

* Unneccesary columns are dropped

* Columns for individual players (4 total) and teams (2 total) are then made

* Data for individual players is downloaded and populated into a Series and placed into DataFrame cells

* Data for teams is downloaded and populated into a Series and placed into DataFrame cells

In [92]:
data = pd.concat([data, new_data(paramaters, head, metadata)]).reset_index(drop=True)
data.tail()

,id,link,rocket_league_id,replay_title,map_code,map_name,playlist_id,playlist_name,duration,overtime,...,uploader.avatar,blue.goals,orange.goals,overtime_seconds,blue_team,blue_player1,blue_player2,orange_team,orange_player1,orange_player2
45,eac4694b-e177-483c-b21e-39a1df7930c6,https://ballchasing.com/api/replays/eac4694b-e...,5000F14D4EDB50872541BAA18749F639,2023-05-16.21.02 NachoBiden Ranked Doubles,cs_p,Champions Field,ranked-doubles,Ranked Doubles,348,False,...,https://avatars.akamai.steamstatic.com/3fa3b23...,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,5e7affa6-ad60-4890-908b-37264a22c43f,https://ballchasing.com/api/replays/5e7affa6-a...,20BE17B04CDD5EA8A79BFE9F134D97EA,2023-05-17.01.02 Jay-510 Ranked Doubles Loss,outlaw_oasis_p,NaN,ranked-doubles,Ranked Doubles,386,True,...,https://avatars.akamai.steamstatic.com/1ed54aa...,2.0,3.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN
47,05d26732-7b5b-4f7f-a4df-3b060d59b3c6,https://ballchasing.com/api/replays/05d26732-7...,2D9353EC4F374281B3FEC88F83DB1AA2,2023-05-16.22.02 Colin Robinson Ranked Doubles...,park_p,Beckwith Park,ranked-doubles,Ranked Doubles,409,False,...,https://steamcdn-a.akamaihd.net/steamcommunity...,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,8211ebcc-0ccc-4528-a896-0dd052bfe7fe,https://ballchasing.com/api/replays/8211ebcc-0...,856CBD534A4B75AC5AAB9593789B2E3D,2023-05-17.01.02 rubywdauziohno Ranked Doubles...,utopiastadium_p,Utopia Coliseum,ranked-doubles,Ranked Doubles,406,False,...,https://avatars.akamai.steamstatic.com/cb8d627...,2.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,52f9f610-b538-4ce3-8adc-0bc9871a5b98,https://ballchasing.com/api/replays/52f9f610-b...,A6DC33B5408B1A4F555F9BA72CA87752,2023-05-17.01.01 Wake Ranked Doubles Win,utopiastadium_lux_p,NaN,ranked-doubles,Ranked Doubles,333,False,...,https://avatars.akamai.steamstatic.com/f6bace2...,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
data['id'].unique()
data[data['id'] == 'd5a78371-fa71-46d7-8b10-11b8a64de5f8']

,id,link,rocket_league_id,replay_title,map_code,map_name,playlist_id,playlist_name,duration,overtime,...,uploader.avatar,blue.goals,orange.goals,overtime_seconds,blue_team,blue_player1,blue_player2,orange_team,orange_player1,orange_player2
46,d5a78371-fa71-46d7-8b10-11b8a64de5f8,https://ballchasing.com/api/replays/d5a78371-f...,BAF69C2646D1BD08FDA27295D260DE26,2023-05-13.23.56 R W 2 Ranked Doubles Loss,eurostadium_p,Mannfield,ranked-doubles,Ranked Doubles,206,False,...,https://avatars.akamai.steamstatic.com/5ed138c...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


id                       object
link                     object
rocket_league_id         object
replay_title             object
map_code                 object
map_name                 object
playlist_id              object
playlist_name            object
duration                  int64
overtime                   bool
season                    int64
season_type              object
date                     object
date_has_tz                bool
visibility               object
created                  object
min_rank.tier             int64
min_rank.division       float64
min_rank.name            object
min_rank.id              object
max_rank.tier             int64
max_rank.division       float64
max_rank.name            object
max_rank.id              object
uploader.steam_id        object
uploader.name            object
uploader.profile_url     object
uploader.avatar          object
blue.goals              float64
orange.goals            float64
overtime_seconds        float64
blue_tea

In [114]:
get_players_csv('40f64128-d24c-4fe3-a959-c6805fb9e29d').T.to_parquet('megabruh.parquet')

In [103]:
get_teams_csv('40f64128-d24c-4fe3-a959-c6805fb9e29d').to_parquet('testseries')

In [40]:
a = add_team_data(data)

In [58]:
b = add_player_data(a)

In [115]:
b.to_parquet() # it would appear that what i need to do is make a player and team object and create a new one each time

ArrowInvalid: ('Could not convert team name                              NaN\ngame duration                       400.00\nscore                               913.00\ngoals                                 4.00\nassists                               1.00\nsaves                                 1.00\nshots                                 7.00\nshots conceded                       12.00\ngoals conceded                        6.00\nshooting percentage                  57.14\nbpm                                 575.00\navg boost amount                     93.76\namount collected                   3974.00\namount collected big pads          2602.00\namount collected small pads        1372.00\ncount collected big pads             31.00\ncount collected small pads          115.00\namount stolen                       629.00\namount stolen big pads              340.00\namount stolen small pads            289.00\ncount stolen big pads                 4.00\ncount stolen small pads              27.00

In [93]:
data['overtime_seconds'].max()

176.0

In [102]:
data.to_parquet('testsave')

In [135]:
players = pd.read_csv('46a5bb4b-49e9-4a47-9150-0888c02864b6-players.csv')
players.columns
for name in players.columns:
    name = name.replace(' ', '_')
    print(f'{name}: uint8')

color: uint8
team_name: uint8
player_name: uint8
platform: uint8
player_id: uint8
car_id: uint8
car_name: uint8
game_duration: uint8
score: uint8
goals: uint8
assists: uint8
saves: uint8
shots: uint8
shots_conceded: uint8
goals_conceded: uint8
goals_conceded_while_last_defender: uint8
shooting_percentage: uint8
bpm: uint8
avg_boost_amount: uint8
amount_collected: uint8
amount_collected_big_pads: uint8
amount_collected_small_pads: uint8
count_collected_big_pads: uint8
count_collected_small_pads: uint8
amount_stolen: uint8
amount_stolen_big_pads: uint8
amount_stolen_small_pads: uint8
count_stolen_big_pads: uint8
count_stolen_small_pads: uint8
0_boost_time: uint8
100_boost_time: uint8
amount_used_while_supersonic: uint8
amount_overfill_total: uint8
amount_overfill_stolen: uint8
avg_speed: uint8
total_distance: uint8
time_slow_speed: uint8
percentage_slow_speed: uint8
time_boost_speed: uint8
percentage_boost_speed: uint8
time_supersonic_speed: uint8
percentage_supersonic_speed: uint8
time_

In [149]:
np.finfo(np.float16)

finfo(resolution=0.001, min=-6.55040e+04, max=6.55040e+04, dtype=float16)